#  Direcotry

In [ ]:
ls

In [39]:
cd dataset

[WinError 2] 系统找不到指定的文件。: 'dataset'
C:\Users\12266\DATATHON\dataset


In [43]:
#import related packages
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np

# Load Data
## Police District

In [44]:
df_police_districts = pd.read_csv("Traffic, Investigations _ Other/police_districts.csv")
df_police_districts.head()

,OBJECTID,PERIMETER,DISTRICT_NUM,LOCATION,LAT,LNG,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
0,343,69282.58846,16,39th St. & Lancaster Ave.,39.96158,-75.19940,SWPD,1.216700e+08,1.927226e+07,27575.07918
1,344,33150.15496,17,20th St. & Federal St.,39.93728,-75.17670,SPD,5.786368e+07,9.154950e+06,13179.95335
2,345,54403.93004,18,55th St. & Pine St.,39.95426,-75.23217,SWPD,9.881929e+07,1.564117e+07,21629.61069
3,346,51597.05160,35,N. Broad St. & Champlost St.,40.04442,-75.14351,NWPD,1.544309e+08,2.450685e+07,20554.86914
4,347,58075.01444,39,22nd St. & Hunting Park Ave.,40.01362,-75.15884,NWPD,1.578976e+08,2.503787e+07,23123.58026


## Crime with District

In [45]:
with open('Data Open 2023 Additional Data/philadelphia.geojson') as handle:
    philly_geo = json.loads(handle.read())


crime = pd.read_csv("Traffic, Investigations _ Other/crimes.csv")
district_gdf = gpd.read_file('Data Open 2023 Additional Data/Boundaries_District.geojson')

# create Geom column in crime_df using lat and lon
geometry = [Point(xy) for xy in zip(crime['lng'], crime['lat'])]
crime_gdf = gpd.GeoDataFrame(crime, crs="EPSG:4326", geometry=geometry)
crime_with_district = gpd.sjoin(crime_gdf, district_gdf, how='left', op='within')
crime_with_district = crime_with_district.dropna(subset=["DIST_NUM"])
crime_with_district.head(), len(crime_with_district)

(   objectid psa dispatch_date dispatch_time         location_block  \
 0    218891   2    2022-09-08      15:21:00  3200 BLOCK N BROAD ST   
 1    218892   2    2022-09-14      17:41:00  3200 BLOCK N BROAD ST   
 2    218893   2    2022-09-19      15:23:00  3200 BLOCK N BROAD ST   
 3    218894   2    2022-08-22      09:38:00        0 BLOCK WOLF ST   
 4    218895   2    2022-08-20      10:31:00        0 BLOCK WOLF ST   
 
     text_general_code        lat        lng          fips  \
 0              Thefts  40.002741 -75.153068  4.210102e+11   
 1              Thefts  40.002741 -75.153068  4.210102e+11   
 2              Thefts  40.002741 -75.153068  4.210102e+11   
 3  Theft from Vehicle  39.918351 -75.145999  4.210198e+11   
 4  Theft from Vehicle  39.918351 -75.145999  4.210198e+11   
 
                      geometry  ...  DISTRICT_ID  DIST_NUM SUM_AREA  DIST_NUMC  \
 0  POINT (-75.15307 40.00274)  ...         None      39.0     None         39   
 1  POINT (-75.15307 40.00274)  ..

In [46]:
crime_with_district["dispatch_date"].max(),crime_with_district["dispatch_date"].min() 

('2022-12-31', '2012-12-31')

## Crash with District

In [47]:
df_crash = pd.read_csv("Crashes/crash_info_general.csv")

district_gdf = gpd.read_file('Data Open 2023 Additional Data/Boundaries_District.geojson')

# create Geom column in crime_df using lat and lon
geometry = [Point(xy) for xy in zip(df_crash['DEC_LONG'], df_crash['DEC_LAT'])]
crash_gdf = gpd.GeoDataFrame(df_crash, crs="EPSG:4326", geometry=geometry)
crash_with_district = gpd.sjoin(crash_gdf, district_gdf, how='left', op='within')
crash_with_district = crash_with_district.dropna(subset=["DIST_NUM"])
print(len(crash_with_district))
crash_with_district.head()

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (97,98,99) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


131569


,Unnamed: 0,CRN,ARRIVAL_TM,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,BICYCLE_COUNT,BICYCLE_DEATH_COUNT,BICYCLE_SUSP_SERIOUS_INJ_COUNT,BUS_COUNT,...,DISTRICT_ID,DIST_NUM,SUM_AREA,DIST_NUMC,LOCATION,PHONE,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
1,2,2010004457,658.0,2,0,0,0,0,0,0,...,None,6.0,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692
2,3,2010005005,1217.0,2,0,0,0,0,0,0,...,None,6.0,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692
4,5,2010005373,36.0,0,0,0,0,0,0,0,...,None,16.0,None,16,39th St. & Lancaster Ave.,686-3160,SWPD,1.216700e+08,1.927226e+07,27575.079183
5,6,2010005374,326.0,1,0,0,0,0,0,0,...,None,15.0,None,15,Harbison Ave. & Levick St.,686-3150,NEPD,3.077864e+08,4.881580e+07,35944.018281
6,7,2010005375,505.0,1,0,0,0,0,0,0,...,None,3.0,None,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793


## Traffic Stop Data

In [48]:
traffic_stops = pd.read_csv("Traffic, Investigations _ Other/traffic_stops_philadelphia.csv")
traffic_stops.head()

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,objectid,date,time,location,lat,lng,district,service_area,subject_age,subject_race,...,outcome,contraband_found,frisk_performed,search_conducted,search_person,search_vehicle,raw_race,raw_individual_contraband,raw_vehicle_contraband,fips
0,1,2014-01-01,01:14:00,NaN,NaN,NaN,19.0,191,31.0,black,...,arrest,True,False,True,True,False,Black - Non-Latino,True,False,NaN
1,2,2014-01-01,01:57:00,NaN,NaN,NaN,12.0,121,21.0,black,...,arrest,False,True,True,True,False,Black - Non-Latino,False,False,NaN
2,3,2014-01-01,03:30:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,24.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
3,4,2014-01-01,03:40:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,20.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
4,5,2014-01-01,08:30:00,N 56TH ST / UPLAND WAY,39.983712,-75.234188,19.0,193,31.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210101e+11


# Merge Crime, Crash, Traffic Stop

## Deal with crime data

In [49]:
#split crime_with_district into different year_month units
crime_with_district['dispatch_date'] = pd.to_datetime(crime_with_district['dispatch_date'])
crime_with_district['year_month'] = crime_with_district['dispatch_date'].dt.to_period('M')

In [50]:
#### Only the text_general_code will be useful
crime_crosstab = pd.crosstab(index = [crime_with_district['year_month'],crime_with_district['DIST_NUM']], columns = crime_with_district['text_general_code'])
crime_crosstab.columns = ["crime_"+i for i in crime_crosstab.columns]
crime_crosstab['crime_total'] = crime_crosstab.sum(axis=1)
crime_crosstab

crime_Aggravated Assault Firearm  \
year_month DIST_NUM                                     
2012-12    1.0                                      0   
           2.0                                      0   
           3.0                                      0   
           5.0                                      0   
           6.0                                      0   
...                                               ...   
2022-12    25.0                                    37   
           26.0                                    11   
           35.0                                    19   
           39.0                                    16   
           77.0                                     0   

                     crime_Aggravated Assault No Firearm  \
year_month DIST_NUM                                        
2012-12    1.0                                         0   
           2.0                                         0   
           3.0                                         0   
           5.0                                         0   
           6.0                                         0   
...                                                  ...   
2022-12    25.0                                       42   
           26.0                                       14   
           35.0                                       26   
           39.0                                       30   
           77.0                                        0   

                     crime_All Other Offenses  crime_Arson  \
year_month DIST_NUM                                          
2012-12    1.0                              0            0   
           2.0                              0            0   
           3.0                              0            0   
           5.0                              0            0   
           6.0                              0            0   
...                                       ...          ...   
2022-12    25.0                            60            4   
           26.0                            28            1   
           35.0                            59            0   
           39.0                            46            0   
           77.0                             2            0   

                     crime_Burglary Non-Residential  \
year_month DIST_NUM                                   
2012-12    1.0                                    0   
           2.0                                    0   
           3.0                                    0   
           5.0                                    0   
           6.0                                    0   
...                                             ...   
2022-12    25.0                                   7   
           26.0                                  23   
           35.0                                   2   
           39.0                                  12   
           77.0                                   0   

                     crime_Burglary Residential  \
year_month DIST_NUM                               
2012-12    1.0                                0   
           2.0                                0   
           3.0                                0   
           5.0                                0   
           6.0                                0   
...                                         ...   
2022-12    25.0                               9   
           26.0                              28   
           35.0                              16   
           39.0                              20   
           77.0                               0   

                     crime_DRIVING UNDER THE INFLUENCE  \
year_month DIST_NUM                                      
2012-12    1.0                                       0   
           2.0                                       0   
           3.0                                       2   
           5.0       

we conduct the transformation of log(1+x) to all the counting numbers. For counting features, logarithmic transformation is usually a good preprocessing method. This is because count-type features may have a long-tailed distribution, and logarithmic transformation helps reduce the influence of outliers and make the data closer to the normal distribution.

In [51]:
crime=np.log1p(crime_crosstab["crime_total"])
crime

year_month  DIST_NUM
2012-12     1.0         1.098612
            2.0         1.609438
            3.0         1.791759
            5.0         0.693147
            6.0         1.386294
                          ...   
2022-12     25.0        6.626718
            26.0        6.272877
            35.0        6.669498
            39.0        6.514713
            77.0        3.850148
Name: crime_total, Length: 2660, dtype: float64

## Deal with crash data

In [65]:
number_of_crashes = crash_with_district.groupby("DIST_NUM").count()["CRN"]
number_of_crashes.head()
crash_with_district["URBAN_RURAL"].nunique()

1

In [66]:
crash_with_district.filter(like='ROAD_CONDITION').value_counts() 

ROAD_CONDITION
1                 104516
9                  22633
2                   1409
7                    882
98                   699
6                    610
99                   384
8                    380
0                     34
3                     11
4                      9
5                      2
dtype: int64

In [67]:
#split crash_with_district into different year_month units
crash_with_district['date_str'] = crash_with_district['CRASH_YEAR'].astype(str) + '-' + crash_with_district['CRASH_MONTH'].astype(str).str.zfill(2)
crash_with_district['year_month'] = pd.to_datetime(crash_with_district['date_str']).dt.to_period('M')
crash_with_district = crash_with_district.drop('date_str', axis=1)
crash_with_district.head()

,Unnamed: 0,CRN,ARRIVAL_TM,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,BICYCLE_COUNT,BICYCLE_DEATH_COUNT,BICYCLE_SUSP_SERIOUS_INJ_COUNT,BUS_COUNT,...,DIST_NUM,SUM_AREA,DIST_NUMC,LOCATION,PHONE,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length,year_month
1,2,2010004457,658.0,2,0,0,0,0,0,0,...,6.0,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692,2010-01
2,3,2010005005,1217.0,2,0,0,0,0,0,0,...,6.0,None,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692,2010-01
4,5,2010005373,36.0,0,0,0,0,0,0,0,...,16.0,None,16,39th St. & Lancaster Ave.,686-3160,SWPD,1.216700e+08,1.927226e+07,27575.079183,2010-01
5,6,2010005374,326.0,1,0,0,0,0,0,0,...,15.0,None,15,Harbison Ave. & Levick St.,686-3150,NEPD,3.077864e+08,4.881580e+07,35944.018281,2010-01
6,7,2010005375,505.0,1,0,0,0,0,0,0,...,3.0,None,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793,2010-01


We will keep the following features

categorical feat: *_TYPE, ILLUMINATION,  RELATION_TO_ROAD, ROAD_CONDITION, URBAN_RURAL, WEATHER1

numerical feat: *_COUNT of drivers in different age groups

date feat: YEAR_MONTH

In [68]:
tmp=crash_with_district

In [69]:
manual_cat_list = ["ILLUMINATION", "ROAD_CONDITION", "WEATHER1"]
manual_drop_list = ["RDWY_SURF_TYPE_CD", "WORK_ZONE_TYPE","COLLISION_TYPE","TCD_TYPE"]
crash_useful_cat_feat_list = [i for i in crash_with_district.columns if ("_TYPE" in i and i not in manual_drop_list) or i in manual_cat_list]
crash_useful_num_feat_list = ["VEHICLE_COUNT","DEC_LONG","DEC_LAT"] #add long lat additionally
crash_useful_cat_feat=pd.get_dummies(crash_with_district[crash_useful_cat_feat_list].astype(str),prefix=crash_useful_cat_feat_list)
crash_useful_feat = pd.concat([crash_with_district[crash_useful_num_feat_list], crash_useful_cat_feat], axis=1)
crash_useful_feat["year_month"]=crash_with_district["year_month"]
crash_useful_feat["DIST_NUM"]=crash_with_district["DIST_NUM"]

In [70]:
crash_useful_feat.filter(like='INTERSECT_TYPE')

,INTERSECT_TYPE_0,INTERSECT_TYPE_1,INTERSECT_TYPE_10,INTERSECT_TYPE_11,INTERSECT_TYPE_12,INTERSECT_TYPE_2,INTERSECT_TYPE_3,INTERSECT_TYPE_5,INTERSECT_TYPE_6,INTERSECT_TYPE_7,INTERSECT_TYPE_9
1,1,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
133008,1,0,0,0,0,0,0,0,0,0,0
133009,0,0,0,0,0,0,0,1,0,0,0
133010,1,0,0,0,0,0,0,0,0,0,0
133011,0,1,0,0,0,0,0,0,0,0,0


In [71]:
crash_useful_feat

,VEHICLE_COUNT,DEC_LONG,DEC_LAT,ILLUMINATION_1.0,ILLUMINATION_2.0,ILLUMINATION_3.0,ILLUMINATION_4.0,ILLUMINATION_5.0,ILLUMINATION_6.0,ILLUMINATION_8.0,...,WEATHER1_4,WEATHER1_5,WEATHER1_6,WEATHER1_7,WEATHER1_8,WEATHER1_9,WEATHER1_98,WEATHER1_99,year_month,DIST_NUM
1,3,-75.1433,39.9423,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,2010-01,6.0
2,3,-75.1575,39.9590,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2010-01,6.0
4,2,-75.2075,39.9822,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,2010-01,16.0
5,1,-75.0846,39.9995,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2010-01,15.0
6,1,-75.1516,39.9018,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2010-01,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133008,2,-75.2275,39.9776,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2021-07,19.0
133009,2,-75.1495,39.9547,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,2021-11,6.0
133010,3,-75.1220,40.0023,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2021-07,25.0
133011,2,-75.1630,39.9903,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2021-11,22.0


In [72]:
crash_useful_feat.columns 

Index(['VEHICLE_COUNT', 'DEC_LONG', 'DEC_LAT', 'ILLUMINATION_1.0',
       'ILLUMINATION_2.0', 'ILLUMINATION_3.0', 'ILLUMINATION_4.0',
       'ILLUMINATION_5.0', 'ILLUMINATION_6.0', 'ILLUMINATION_8.0',
       'ILLUMINATION_9.0', 'ILLUMINATION_nan', 'INTERSECT_TYPE_0',
       'INTERSECT_TYPE_1', 'INTERSECT_TYPE_10', 'INTERSECT_TYPE_11',
       'INTERSECT_TYPE_12', 'INTERSECT_TYPE_2', 'INTERSECT_TYPE_3',
       'INTERSECT_TYPE_5', 'INTERSECT_TYPE_6', 'INTERSECT_TYPE_7',
       'INTERSECT_TYPE_9', 'LOCATION_TYPE_0', 'LOCATION_TYPE_1',
       'LOCATION_TYPE_2', 'LOCATION_TYPE_3', 'LOCATION_TYPE_4',
       'LOCATION_TYPE_5', 'LOCATION_TYPE_6', 'LOCATION_TYPE_7',
       'LOCATION_TYPE_8', 'LOCATION_TYPE_99', 'ROAD_CONDITION_0',
       'ROAD_CONDITION_1', 'ROAD_CONDITION_2', 'ROAD_CONDITION_3',
       'ROAD_CONDITION_4', 'ROAD_CONDITION_5', 'ROAD_CONDITION_6',
       'ROAD_CONDITION_7', 'ROAD_CONDITION_8', 'ROAD_CONDITION_9',
       'ROAD_CONDITION_98', 'ROAD_CONDITION_99', 'WEATHER1_10', 'WEA

In [73]:
crash_useful_feat_group=crash_useful_feat

## Deal with traffic_stops data

In [74]:
traffic_stops.head()

,objectid,date,time,location,lat,lng,district,service_area,subject_age,subject_race,...,outcome,contraband_found,frisk_performed,search_conducted,search_person,search_vehicle,raw_race,raw_individual_contraband,raw_vehicle_contraband,fips
0,1,2014-01-01,01:14:00,NaN,NaN,NaN,19.0,191,31.0,black,...,arrest,True,False,True,True,False,Black - Non-Latino,True,False,NaN
1,2,2014-01-01,01:57:00,NaN,NaN,NaN,12.0,121,21.0,black,...,arrest,False,True,True,True,False,Black - Non-Latino,False,False,NaN
2,3,2014-01-01,03:30:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,24.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
3,4,2014-01-01,03:40:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,20.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
4,5,2014-01-01,08:30:00,N 56TH ST / UPLAND WAY,39.983712,-75.234188,19.0,193,31.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210101e+11


In [75]:
# cat feat: subject race, subject sex, type, arrest_made, outcome, contraband_found, frisk_performed, search_conducted

traffic_stop_useful_cat_feat = traffic_stops[["date","district","subject_race", "subject_sex", "type", "arrest_made", "outcome", "contraband_found", "frisk_performed", "search_conducted"]]
traffic_stop_useful_cat_feat["outcome"] = traffic_stop_useful_cat_feat['outcome'].fillna("none")
traffic_stop_useful_cat_feat["contraband_found"] = traffic_stop_useful_cat_feat['contraband_found'].fillna(False).astype(bool)
traffic_stop_useful_cat_feat

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,date,district,subject_race,subject_sex,type,arrest_made,outcome,contraband_found,frisk_performed,search_conducted
0,2014-01-01,19.0,black,male,pedestrian,True,arrest,True,False,True
1,2014-01-01,12.0,black,male,pedestrian,True,arrest,False,True,True
2,2014-01-01,18.0,black,male,pedestrian,False,none,False,False,False
3,2014-01-01,18.0,black,male,pedestrian,False,none,False,False,False
4,2014-01-01,19.0,black,male,vehicular,False,none,False,False,False
...,...,...,...,...,...,...,...,...,...,...
1865091,2018-04-14,12.0,black,male,pedestrian,False,none,False,False,False
1865092,2018-04-14,39.0,asian/pacific islander,male,pedestrian,False,none,False,False,False
1865093,2018-04-14,35.0,black,male,vehicular,True,arrest,False,True,True
1865094,2018-04-14,35.0,black,male,vehicular,True,arrest,False,True,True


In [76]:
#split traffic_stops into different year_month units
traffic_stop_useful_cat_feat['date'] = pd.to_datetime(traffic_stop_useful_cat_feat['date'])
traffic_stop_useful_cat_feat['year_month'] = traffic_stop_useful_cat_feat['date'].dt.to_period('M')

C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\envs\aaa\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [77]:
number_of_traffic_stops = pd.get_dummies(traffic_stop_useful_cat_feat).groupby(["year_month","district"]).count()['arrest_made']
number_of_traffic_stops

year_month  district
2014-01     1.0          382
            2.0          304
            3.0          176
            5.0           47
            6.0          147
                        ... 
2018-04     25.0         947
            26.0         293
            35.0        1364
            39.0        1070
            77.0          35
Name: arrest_made, Length: 1144, dtype: int64

In [78]:
traffic_stop_sum = pd.get_dummies(traffic_stop_useful_cat_feat).groupby(["year_month","district"]).sum()
traffic_stop_sum.columns = ['traffic_stop_'+i+'_count' for i in traffic_stop_sum.columns]
traffic_stop_sum['traffic_stop_total'] = number_of_traffic_stops
traffic_stop_sum

traffic_stop_arrest_made_count  \
year_month district                                   
2014-01    1.0                                   12   
           2.0                                   19   
           3.0                                    4   
           5.0                                    0   
           6.0                                   14   
...                                             ...   
2018-04    25.0                                  39   
           26.0                                  21   
           35.0                                  70   
           39.0                                  49   
           77.0                                   5   

                     traffic_stop_contraband_found_count  \
year_month district                                        
2014-01    1.0                                         2   
           2.0                                         5   
           3.0                                         0   
           5.0                                         1   
           6.0                                         8   
...                                                  ...   
2018-04    25.0                                       16   
           26.0                                        6   
           35.0                                       19   
           39.0                                       18   
           77.0                                        2   

                     traffic_stop_frisk_performed_count  \
year_month district                                       
2014-01    1.0                                       23   
           2.0                                       28   
           3.0                                       11   
           5.0                                        2   
           6.0                                       13   
...                                                 ...   
2018-04    25.0                                     100   
           26.0                                       9   
           35.0                                     144   
           39.0                                     111   
           77.0                                       2   

                     traffic_stop_search_conducted_count  \
year_month district                                        
2014-01    1.0                                        14   
           2.0                                        12   
           3.0                                         4   
           5.0                                         2   
           6.0                                        14   
...                                                  ...   
2018-04    25.0                                       86   
           26.0                                       29   
           35.0                                      161   
           39.0                                       78   
           77.0                                       32   

                     traffic_stop_subject_race_asian/pacific islander_count  \
year_month district                                                           
2014-01    1.0                                                    28.0        
           2.0                                                    17.0        
           3.0                                                    17.0        
           5.0                                                     0.0        
           6.0                                                    15.0        
...                                                                ...        
2018-04    25.0                                                    5.0        
           26.0                                                    9.0        
           35.0                                                   23.0        
           39.0                                                    7.0        
           77.0            

In [79]:
#we conduct the transformation of log(1+x)
traffic_stop_transformed=np.log1p(traffic_stop_sum)
traffic_stop=traffic_stop_transformed["traffic_stop_total"]
traffic_stop

year_month  district
2014-01     1.0         5.948035
            2.0         5.720312
            3.0         5.176150
            5.0         3.871201
            6.0         4.997212
                          ...   
2018-04     25.0        6.854355
            26.0        5.683580
            35.0        7.218910
            39.0        6.976348
            77.0        3.583519
Name: traffic_stop_total, Length: 1144, dtype: float64

### Merge

In [80]:
crash_useful_feat_group

,VEHICLE_COUNT,DEC_LONG,DEC_LAT,ILLUMINATION_1.0,ILLUMINATION_2.0,ILLUMINATION_3.0,ILLUMINATION_4.0,ILLUMINATION_5.0,ILLUMINATION_6.0,ILLUMINATION_8.0,...,WEATHER1_4,WEATHER1_5,WEATHER1_6,WEATHER1_7,WEATHER1_8,WEATHER1_9,WEATHER1_98,WEATHER1_99,year_month,DIST_NUM
1,3,-75.1433,39.9423,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,2010-01,6.0
2,3,-75.1575,39.9590,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2010-01,6.0
4,2,-75.2075,39.9822,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,2010-01,16.0
5,1,-75.0846,39.9995,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2010-01,15.0
6,1,-75.1516,39.9018,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2010-01,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133008,2,-75.2275,39.9776,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2021-07,19.0
133009,2,-75.1495,39.9547,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,2021-11,6.0
133010,3,-75.1220,40.0023,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2021-07,25.0
133011,2,-75.1630,39.9903,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2021-11,22.0


In [81]:
df_merged = pd.merge(crime.reset_index(), crash_useful_feat_group.reset_index(), on=['year_month','DIST_NUM'])
df_merged=pd.merge(df_merged,traffic_stop.reset_index().rename(columns={'district': 'DIST_NUM'}), on=['year_month','DIST_NUM'])
#here are less rows than traffic_stop_standardized dataset above, which is weired

In [82]:
df_merged=df_merged.drop(["index"],axis=1)

In [84]:
df_merged.to_csv("merged_data_onehot.csv")

In [85]:
df_merged

,year_month,DIST_NUM,crime_total,VEHICLE_COUNT,DEC_LONG,DEC_LAT,ILLUMINATION_1.0,ILLUMINATION_2.0,ILLUMINATION_3.0,ILLUMINATION_4.0,...,WEATHER1_3,WEATHER1_4,WEATHER1_5,WEATHER1_6,WEATHER1_7,WEATHER1_8,WEATHER1_9,WEATHER1_98,WEATHER1_99,traffic_stop_total
0,2014-01,1.0,5.416100,2,-75.1991,39.9309,0,0,1,0,...,1,0,0,0,0,0,0,0,0,5.948035
1,2014-01,1.0,5.416100,2,-75.1991,39.9309,0,0,1,0,...,0,0,0,0,0,0,0,0,0,5.948035
2,2014-01,1.0,5.416100,1,-75.1898,39.9191,1,0,0,0,...,1,0,0,0,0,0,0,0,0,5.948035
3,2014-01,1.0,5.416100,1,-75.1879,39.9142,1,0,0,0,...,0,0,0,0,0,0,0,0,0,5.948035
4,2014-01,1.0,5.416100,2,-75.1968,39.8930,1,0,0,0,...,1,0,0,0,0,0,0,0,0,5.948035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48314,2018-04,39.0,6.514713,4,-75.1580,40.0101,0,0,1,0,...,0,0,0,0,1,0,0,0,0,6.976348
48315,2018-04,39.0,6.514713,1,-75.1837,40.0306,1,0,0,0,...,1,0,0,0,0,0,0,0,0,6.976348
48316,2018-04,39.0,6.514713,1,-75.1906,40.0009,1,0,0,0,...,1,0,0,0,0,0,0,0,0,6.976348
48317,2018-04,39.0,6.514713,1,-75.1723,40.0079,1,0,0,0,...,1,0,0,0,0,0,0,0,0,6.976348


In [86]:
df_merged.head()

,year_month,DIST_NUM,crime_total,VEHICLE_COUNT,DEC_LONG,DEC_LAT,ILLUMINATION_1.0,ILLUMINATION_2.0,ILLUMINATION_3.0,ILLUMINATION_4.0,...,WEATHER1_3,WEATHER1_4,WEATHER1_5,WEATHER1_6,WEATHER1_7,WEATHER1_8,WEATHER1_9,WEATHER1_98,WEATHER1_99,traffic_stop_total
0,2014-01,1.0,5.4161,2,-75.1991,39.9309,0,0,1,0,...,1,0,0,0,0,0,0,0,0,5.948035
1,2014-01,1.0,5.4161,2,-75.1991,39.9309,0,0,1,0,...,0,0,0,0,0,0,0,0,0,5.948035
2,2014-01,1.0,5.4161,1,-75.1898,39.9191,1,0,0,0,...,1,0,0,0,0,0,0,0,0,5.948035
3,2014-01,1.0,5.4161,1,-75.1879,39.9142,1,0,0,0,...,0,0,0,0,0,0,0,0,0,5.948035
4,2014-01,1.0,5.4161,2,-75.1968,39.8930,1,0,0,0,...,1,0,0,0,0,0,0,0,0,5.948035
